<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#分词" data-toc-modified-id="分词-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>分词</a></span><ul class="toc-item"><li><span><a href="#jieba加载多个词典" data-toc-modified-id="jieba加载多个词典-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>jieba加载多个词典</a></span></li></ul></li></ul></div>

## 分词

### jieba加载多个词典

In [1]:
import jieba

In [2]:
# prepare dict1.txt dict2.txt
dict_path_1 = "/tmp/dict_1.txt"
dict_path_2 = "/tmp/dict_2.txt"
!echo "ipv5\nclock" > {dict_path_1}
!echo "Thursday\nmorning" > {dict_path_2}
!cat {dict_path_1}
!cat {dict_path_2}
input_str = "ipv5teighto'clockonThursdayipv4morningtestjie"

ipv5
clock
Thursday
morning


In [3]:
list(jieba.cut(input_str))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.243 seconds.
Prefix dict has been built succesfully.


['ipv5teighto', "'", 'clockonThursdayipv4morningtestjie']

In [4]:
jieba.add_word("jie")
list(jieba.cut(input_str))

['ipv5teighto', "'", 'clockonThursdayipv4morningtest', 'jie']

In [5]:
jieba.load_userdict(dict_path_1)
list(jieba.cut(input_str))

['ipv5', 'teighto', "'", 'clock', 'onThursdayipv4morningtest', 'jie']

In [6]:
jieba.load_userdict(dict_path_2)
list(jieba.cut(input_str))

['ipv5',
 'teighto',
 "'",
 'clock',
 'on',
 'Thursday',
 'ipv4',
 'morning',
 'test',
 'jie']